In [1]:
import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import math
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import re
import os
import sys
from pathlib import Path
path = Path.cwd()
input_dir = path.parent
import cv2
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.transforms import functional as F
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split, StratifiedKFold
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import wandb

In [7]:
train_dir = input_dir/'train'
test_dir = input_dir/'test'
checkpoint_dir = input_dir/"experiment/checkpoint"
cfg = {}

In [8]:
df = pd.read_csv(input_dir/'train.csv')
test_df = pd.read_csv(input_dir/"sample_submission.csv")
annotated_df = pd.read_csv(input_dir/"test_df.csv")

In [9]:
class GlobalWheatDataset(Dataset):
    def __init__(self, df, image_ids, data_dir, transforms, train=True):
        self.df = df
        self.image_ids = image_ids
        self.data_dir = data_dir
        self.transforms = transforms
        self.train = train
    
    def __len__(self,):
        return len(self.image_ids)
    
    def __getitem__(self, index):
        image_id = self.image_ids[index]
        filename = image_id + ".jpg"
        image_path = os.path.join(self.data_dir, filename)
        image = self.load_image(image_path)
        if self.train:
            bboxes_areas = self.df[self.df.image_id==image_id][["xmin", "ymin", "xmax", "ymax", "area"]].values#.astype(np.float)
            #bboxes_areas = torch.as_tensor(bboxes_areas, dtype=torch.float32)  
            bboxes = bboxes_areas[:, :-1]
            area = torch.as_tensor(bboxes_areas[:, -1])
            labels = torch.ones(len(bboxes), dtype=torch.int64)
            image_id = torch.as_tensor([index], dtype=torch.int64)
            iscrowd = torch.zeros(len(bboxes_areas), dtype=torch.uint8)
            
            target = {"boxes": bboxes,
                      "labels": labels,
                      "image_id": image_id,
                      "area": area,
                      "iscrowd": iscrowd,
                      }

            if self.transforms:
                sample = self.transforms(**{"image": image,
                                            "bboxes": bboxes,
                                            "labels": labels})
                
                target["boxes"] = torch.as_tensor(sample["bboxes"], dtype=torch.float32).reshape(-1, 4)
                image = sample["image"]
                return image, target
            else:
                return image, target
        else:
            sample = self.transforms(**{"image": image})
            return sample["image"], image_id
        
    
    def load_image(self, image_path):
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
#         image = Image.open(image_path).convert("RGB")
        return image

def get_test_transforms():
    return A.Compose(
        [
             A.Resize(height=1024, width=1024, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0,
    )


def collate_fn(batch):
    sample = tuple(zip(*batch))
    return sample

test_transforms = get_test_transforms()

def get_test_dataloader():
    test_dataset = GlobalWheatDataset(test_df, test_df.image_id.unique(), test_dir, test_transforms, train=False)
    test_dataloader = DataLoader(test_dataset, batch_size=10, num_workers=4)
    return test_dataloader

In [12]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def load_model():
    # load a model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

    # replace the classifier with a new one, that has
    # num_classes which is user-defined
    num_classes = 2  # 1 class (person) + background
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

def load_checkpoint(model, checkpoint_path=checkpoint_dir/'fasterrcnn_resnet50_fpn_epoch_13_.pth', device="cpu"):
    #checkpoint_path = path.parent/"input/checkpoint2/global_wheat_fasterrcnn_epoch_5.pth"
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint)
    return model

In [14]:
import torchvision
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch.nn as nn


class Resnet50_fpn(nn.Module):
    def __init__(self):
        super(Resnet50_fpn, self).__init__()
        self.model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained_backbone=False)
        num_classes = 2

        # anchor_sizes = ((32,), (64,), (128,), (256,), (512,))
        # aspect_ratios = ((0.5, 1.0, 2.0),) * len(anchor_sizes)
        # rpn_anchor_generator = AnchorGenerator(
        #     anchor_sizes, aspect_ratios
        # )
        inf_features = self.model.roi_heads.box_predictor.cls_score.in_features
        self.model.roi_heads.box_predictor = FastRCNNPredictor(inf_features, num_classes)

    def forward(self, x, y):
        loss_dict = self.model(x, y)
        # print(loss_dict)
        return loss_dict


class Models(object):
    networks = {"resnet50_fpn": Resnet50_fpn()}

    def get_model(self, reference):
        available_models = ", ".join(self.networks.keys())
        assert (reference in self.networks.keys()), "Please choose one of: [{}].".format(available_models)
        return self.networks[reference]


In [15]:
model = Models().get_model("resnet50_fpn")

In [24]:
checkpoint = torch.load(checkpoint_path, map_location=device)["model_state_dict"]
model.load_state_dict(checkpoint)
torch.save(model.state_dict(), checkpoint_dir/"checkpoint_13.pth")

In [10]:
def inference(fold, checkpoint, device):
    test_loader = get_test_dataloader()
    predictions = dict()
    model = load_model()
    model.to(device)
    model = load_checkpoint(model, checkpoint, device)
    model.eval()
    with torch.no_grad():
        
        for image, target in test_loader:
            print(target)
            image = image.to(device)
            preds = model(image)
            for (p, t) in zip(preds, target):
                predictions[t] = p
    return predictions
    

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
checkpoint_path = checkpoint_dir/"fasterrcnn_resnet50_fpn_epoch_13_.pth"
predictions = inference(0, checkpoint=checkpoint_path, device=device)

RuntimeError: Error(s) in loading state_dict for FasterRCNN:
	Missing key(s) in state_dict: "backbone.body.conv1.weight", "backbone.body.bn1.weight", "backbone.body.bn1.bias", "backbone.body.bn1.running_mean", "backbone.body.bn1.running_var", "backbone.body.layer1.0.conv1.weight", "backbone.body.layer1.0.bn1.weight", "backbone.body.layer1.0.bn1.bias", "backbone.body.layer1.0.bn1.running_mean", "backbone.body.layer1.0.bn1.running_var", "backbone.body.layer1.0.conv2.weight", "backbone.body.layer1.0.bn2.weight", "backbone.body.layer1.0.bn2.bias", "backbone.body.layer1.0.bn2.running_mean", "backbone.body.layer1.0.bn2.running_var", "backbone.body.layer1.0.conv3.weight", "backbone.body.layer1.0.bn3.weight", "backbone.body.layer1.0.bn3.bias", "backbone.body.layer1.0.bn3.running_mean", "backbone.body.layer1.0.bn3.running_var", "backbone.body.layer1.0.downsample.0.weight", "backbone.body.layer1.0.downsample.1.weight", "backbone.body.layer1.0.downsample.1.bias", "backbone.body.layer1.0.downsampl

In [ ]:
#predictions = inference(0)
submission = dict()
nboxes = 0
added_boxes = 0
image_ids = []
predictString = []
max_value = 0
for img_id, pred in predictions.items():
        boxes = pred["boxes"].cpu().numpy()#.tolist()
        max_value = max(max_value, boxes.max())
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        boxes = boxes.tolist()

        scores = pred["scores"].cpu().numpy()
        image_ids.append(img_id)
        pred_list = []
        nboxes += len(scores)
        for box, score in zip(boxes, scores):
            
            if score > 0.5:
                x = round(box[0])
                y = round(box[1])
                w = round(box[2])
                h = round(box[3])
#                 max_value = max(max_value, x+w)
#                 max_value = max(max_value, y+h)
                pred_list.append(f"{score} {x} {y} {w} {h}")
                print(f"{score} {x} {y} {w} {h}")
                added_boxes += 1
        
        predictionString = " ".join([str(i) for i in pred_list])
        predictString.append(predictionString)
        
submission = {"image_id": image_ids,
              "PredictionString": predictString}
sub_df = pd.DataFrame.from_dict(submission)
